# Aula 7 - Solução dos exercícios
Leandro Carísio Fernandes

<br>

Implementar a fase de indexação e buscas de um modelo esparso

- Usar este modelo SPLADE já treinado naver/splade_v2_distil (do distilbert) ou splade-cocondenser-selfdistil (do BERT-base 110M params). Mais informações sobre os modelos estão neste artigo: https://arxiv.org/pdf/2205.04733.pdf
- Não é necessário treinar o modelo
- Avaliar nDCG@10 no TREC-COVID e comparar resultados com o BM25 e buscador denso da semana passada
- A dificuldade do exercício está em implementar a função de busca e ranqueamento usada pelo SPLADE. A implementação do índice invertido é apenas um "dicionário python".
- Comparar seus resultados com a busca "original" do SPLADE.
Medir latencia (s/query)


## Preparação do ambiente

In [81]:
!nvidia-smi

Wed Apr 26 00:47:25 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   74C    P0    33W /  70W |   9583MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Variáveis para controlar o fluxo do caderno

In [82]:
# Setar como True só pra gerar a matriz de documentos pela primeira vez
gerar_indice_invertido_docs = True

# Resultados para esse conjunto de parâmetros
# nDCG@10: 0.7269 (fp16) / Latência: 53.9 s para 50 queries
#                           nDCG@10: 0.7282 (fp32) -> calculado em outro caderno
param = {
    'agg': 'max',
    'nome_modelo': 'naver/splade-cocondenser-ensembledistil',
    'manter_contribuicao_CLS_SEP_da_matriz_doc': True,
    'nome_arquivo_indice_invertido_docs': 'idx_splade_naver_splade-cocondenser-ensembledistil_max_com_cls_sep.pickle'
}

# Resultados para esse conjunto de parâmetros
# nDCG@10: 0.7242 / Latência: 52.6 s para 50 queries ()
# param = {
#     'agg': 'max',
#     'nome_modelo': 'naver/splade-cocondenser-ensembledistil',
#     'manter_contribuicao_CLS_SEP_da_matriz_doc': False,
#     'nome_arquivo_indice_invertido_docs': 'idx_splade_naver_splade-cocondenser-ensembledistil_max_sem_cls_sep.pickle'
# }

# Resultados para esse conjunto de parâmetros
# nDCG@10: 0.7103 / Latência: 58.8 s para 50 queries
# param = {
#     'agg': 'max',
#     'nome_modelo': 'naver/splade_v2_distil',
#     'manter_contribuicao_CLS_SEP_da_matriz_doc': True,
#     'nome_arquivo_indice_invertido_docs': 'idx_splade_naver_splade_v2_distil_max_com_cls_sep.pickle'
# }

# Calculando
# nDCG = 0.0655
# param = {
#     'agg': 'sum',
#     'nome_modelo': 'naver/splade_v2_distil',
#     'manter_contribuicao_CLS_SEP_da_matriz_doc': True,
#     'nome_arquivo_indice_invertido_docs': 'idx_splade_naver_splade_v2_distil_sum_com_cls_sep.pickle'
# }

# Resultados para esse conjunto de parâmetros
# nDCG@10: 0.1921 / Latência: 54.8 s para 50 queries (~1,09 seg/query)
# param = {
#     'agg': 'sum',
#     'nome_modelo': 'naver/splade-cocondenser-ensembledistil',
#     'manter_contribuicao_CLS_SEP_da_matriz_doc': True,
#     'nome_arquivo_indice_invertido_docs': 'idx_splade_naver_splade-cocondenser-ensembledistil_sum_com_cls_sep.pickle'
# }

dir_aula_7 = '/content/drive/My Drive/IA368-DD_deep_learning_busca/Aula7_splade/'

batch_size_trec_covid = 32
url_trec_covid = 'https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/trec-covid.zip'

### Instalação de libs e montagem do drive

In [83]:
from google.colab import drive
drive.mount('/content/drive')

!pip install transformers datasets -q
!pip install sentence-transformers -q
!pip install pyserini -q
!pip install faiss-gpu -q

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Download e carga dos documentos e queries do TREC-COVID

In [84]:
%%time
from pathlib import Path
import json

if not Path('./collections/trec-covid.zip').is_file():
  !wget {url_trec_covid} -P collections # type: ignore
  !unzip -o collections/trec-covid.zip -d ./collections # type: ignore

# Converte o qrels que veio no trec-covid.zip pra o formato esperado:
with open('./collections/trec-covid/qrels/test.tsv', 'r') as fin:
  data = fin.read().splitlines(True)
with open('./collections/trec-covid/qrels/test_corrigido.tsv', 'w') as fout:
  for linha in data[1:]:
    campos = linha.split()
    fout.write(f'{campos[0]}\t0\t{campos[1]}\t{campos[2]}\n')

def carrega_corpus_trec_covid():
  retorno = []
  with open('./collections/trec-covid/corpus.jsonl') as corpus:
    for i, line in enumerate(corpus):
      doc = json.loads(line)
      #retorno.append({
      #    'id': doc['_id'],
      #    'doc': f"{doc['title']} {doc['text']}"
      #})
      retorno.append(
          (doc['_id'], f"{doc['title']} {doc['text']}")
      )
      if (i % 10000 == 0):
        print(f'Processado {i} documentos')
    return retorno

def carrega_queries_trec_covid():
  retorno = []
  with open('./collections/trec-covid/queries.jsonl') as queries:
    for line in queries:
      query = json.loads(line)
      # Faz apenas uma pequena tradução de _id para id e text para texto
      retorno.append({'id': query['_id'], 'texto': query['text']})
  return retorno

queries_trec_covid = carrega_queries_trec_covid()
corpus_trec_covid = carrega_corpus_trec_covid()

Processado 0 documentos
Processado 10000 documentos
Processado 20000 documentos
Processado 30000 documentos
Processado 40000 documentos
Processado 50000 documentos
Processado 60000 documentos
Processado 70000 documentos
Processado 80000 documentos
Processado 90000 documentos
Processado 100000 documentos
Processado 110000 documentos
Processado 120000 documentos
Processado 130000 documentos
Processado 140000 documentos
Processado 150000 documentos
Processado 160000 documentos
Processado 170000 documentos
CPU times: user 1.42 s, sys: 162 ms, total: 1.58 s
Wall time: 1.55 s


In [85]:
# Ordena corpus de acordo com o tamanho do texto pra tentar diminuir o tempo
corpus_trec_covid = sorted(corpus_trec_covid, key=lambda x: len(x[1]), reverse=True)

## SPLADE

### Testes convertendo uma string simples

In [86]:
%%time

from transformers import AutoModelForMaskedLM, AutoTokenizer
import torch
from torch.nn.functional import relu

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def carregar_tokenizador_e_modelo(nome):
  tokenizer = AutoTokenizer.from_pretrained(nome)
  model = AutoModelForMaskedLM.from_pretrained(nome).to(device)

  return tokenizer, model

def representacao_esparsa_do_texto(model, tokenizer, texto, add_special_tokens=True, manter_contribuicao_cls_sep=False):
  # Tokeniza o texto
  # Para texto = "The quick brown jumps over the lazy dog",
  # tokens = ['the', 'quick', 'brown', 'jumps', 'over', 'the', 'lazy', 'dog'] (tamanho 8)
  # token_ids = [1996, 4248, 2829, 103, 14523, 2058, 1996, 13971, 3899] (tamanho 8)

  # Roda o modelo
  inputs = tokenizer(texto, add_special_tokens=add_special_tokens,
                     return_special_tokens_mask=True,
                     return_tensors='pt',
                     truncation=True,
                     max_length=256)

  with torch.autocast(device_type=str(device), dtype=torch.float16, enabled=True):
    with torch.no_grad():
      outputs = model(input_ids=inputs['input_ids'].to(device), attention_mask=inputs['attention_mask'].to(device))

  # Acessa os logits  
  # outputs.logits.size() = torch.Size([1, 8, 30522])
  # logits.size() = torch.Size([8, 30588])
  logits = outputs.logits[0, :]

  # Pelo artigo, agora a gente calcula somatório [ log(1 + ReLU(w_ij)) ]
  # relu(logits) vai manter o mesmo tamanho: [8, 30588]
  # 1 + relu(logits) também vai manter o mesmo tamanho: [8, 30588]
  # log(1 + relu(logits)) também vai manter o mesmo tamanho: [8, 30588]
  # Feito isso, calcula o somatório na dim=0, o que vai gerar um vetor de tamanho
  # 30588, que é o tamanho do vocabulário:
      
  mask_tokens_validos = 1 - inputs['special_tokens_mask'].to(device)
  mask = mask_tokens_validos.squeeze().unsqueeze(-1).expand(logits.size())

  if manter_contribuicao_cls_sep:
    mask = torch.ones(mask.size()).to(device) # Como não tem batch envolvido, isso é o mesmo que a attention mask

  if param['agg'] == 'sum':
    wj = torch.sum(torch.log(1 + relu(logits*mask)), dim=0)
  else:
    wj, _ = torch.max(torch.log(1 + relu(logits*mask)), dim=0)

  # Agora temos que armazenar esse vetor de forma esparsa...
  return wj.to_sparse()

def converte_token_ids_para_tokens(tokenizer, ids_tokens):
  return tokenizer.convert_ids_to_tokens(ids_tokens)

CPU times: user 124 µs, sys: 0 ns, total: 124 µs
Wall time: 130 µs


In [87]:
tokenizer, model = carregar_tokenizador_e_modelo(param['nome_modelo'])
model.eval()

BertForMaskedLM(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

Testa "where eat pizza":

In [88]:
wj = representacao_esparsa_do_texto(model, tokenizer, "Where eat pizza")
ids = wj.indices()
valores = wj.values()
tokens = converte_token_ids_para_tokens(tokenizer, ids[0])
list(zip(tokens, valores))

[('that', tensor(0.0597, device='cuda:0', dtype=torch.float16)),
 ('where', tensor(0.5859, device='cuda:0', dtype=torch.float16)),
 ('place', tensor(0.8374, device='cuda:0', dtype=torch.float16)),
 ('best', tensor(0.0039, device='cuda:0', dtype=torch.float16)),
 ('country', tensor(0.7222, device='cuda:0', dtype=torch.float16)),
 ('food', tensor(0.6934, device='cuda:0', dtype=torch.float16)),
 ('places', tensor(0.5732, device='cuda:0', dtype=torch.float16)),
 ('culture', tensor(0.1536, device='cuda:0', dtype=torch.float16)),
 ('location', tensor(1.4600, device='cuda:0', dtype=torch.float16)),
 ('famous', tensor(0.3958, device='cuda:0', dtype=torch.float16)),
 ('hotel', tensor(0.4551, device='cuda:0', dtype=torch.float16)),
 ('farm', tensor(0.1238, device='cuda:0', dtype=torch.float16)),
 ('visit', tensor(0.0439, device='cuda:0', dtype=torch.float16)),
 ('website', tensor(0.0010, device='cuda:0', dtype=torch.float16)),
 ('headquarters', tensor(0.3276, device='cuda:0', dtype=torch.float16

In [89]:
wj = representacao_esparsa_do_texto(model, tokenizer, "what about the weather today")
ids = wj.indices()
valores = wj.values()
tokens = converte_token_ids_para_tokens(tokenizer, ids[0])
list(zip(tokens, valores))

[('about', tensor(0.5698, device='cuda:0', dtype=torch.float16)),
 ('now', tensor(1.8574, device='cuda:0', dtype=torch.float16)),
 ('world', tensor(0.3521, device='cuda:0', dtype=torch.float16)),
 ('day', tensor(0.1946, device='cuda:0', dtype=torch.float16)),
 ('england', tensor(0.0241, device='cuda:0', dtype=torch.float16)),
 ('summer', tensor(0.6514, device='cuda:0', dtype=torch.float16)),
 ('today', tensor(2.9023, device='cuda:0', dtype=torch.float16)),
 ('science', tensor(0.5337, device='cuda:0', dtype=torch.float16)),
 ('event', tensor(0.2983, device='cuda:0', dtype=torch.float16)),
 ('current', tensor(0.6099, device='cuda:0', dtype=torch.float16)),
 ('earth', tensor(0.0698, device='cuda:0', dtype=torch.float16)),
 ('nature', tensor(0.0734, device='cuda:0', dtype=torch.float16)),
 ('winter', tensor(0.9053, device='cuda:0', dtype=torch.float16)),
 ('joe', tensor(0.0743, device='cuda:0', dtype=torch.float16)),
 ('storm', tensor(0.7231, device='cuda:0', dtype=torch.float16)),
 ('envi

### Testes em batch

In [90]:
from torch.utils import data
from torch.utils.data import DataLoader

# Definição do Dataset
class Dataset(data.Dataset):
    # Recebe apenas um vetor de textos
    def __init__(self, tokenizer, textos, max_seq_length):
        self.max_seq_length = max_seq_length
        self.tokenizer = tokenizer
        self.textos = textos

    def __len__(self):
        return len(self.textos)
    
    def __getitem__(self, idx):
        # Aqui é só uma passada no eval, não precisa de cache
        item = self.tokenizer(self.textos[idx],
                       padding=True,
                       return_special_tokens_mask=True,
                       # No exemplo dos autores eles não removem o CLS/SEP
                       # https://github.com/naver/splade/blob/main/inference_splade.ipynb
                       add_special_tokens=True, 
                       truncation=True,
                       max_length=self.max_seq_length
                )
        return item

In [91]:
from transformers import BatchEncoding

def collate_fn(batch):
    return BatchEncoding(tokenizer.pad(batch, return_tensors='pt'))

In [92]:
from tqdm.auto import tqdm

def representacao_esparsa_dataloader(model, tokenizer, dataloader, func_executar_apos_batch = lambda idx_batch, wj_batch : None):
  # Ideia do Marcus Piau de usar f16 - otimiza o cálculo
  with torch.autocast(device_type=str(device), dtype=torch.float16, enabled=True):
    with torch.no_grad():
      for i_batch, batch in enumerate(tqdm(dataloader)):
        outputs = model(input_ids = batch['input_ids'].to(device), attention_mask = batch['attention_mask'].to(device))
        logits = outputs.logits

        # Na hora de recuperar os logits, temos duas opções. Ou consideramos só a attention mask (ou seja, 
        # exclui apenas os PAD) ou consideramos a special tokens mask. Nessa situação remove (CLS, SEP e PAD).
        # Deixa configurável pra testar os dois
              # OBS.: No modelo do autor ele usa special_tokens e só remove o attention_mask. Vamos 
              # Mas tem uma classe SpaceDoc que parece que depois tira manualmente. Então vamos tentar simular
              # tirando os special_tokens_mask e depois se não der certo, tentamos tirar só o attention_mask
              # implementar assim também então: https://github.com/naver/splade/blob/main/splade/models/transformer_rep.py
        if param['manter_contribuicao_CLS_SEP_da_matriz_doc']:
          mask_tokens_validos = batch['attention_mask'].to(device)
        else:
          mask_tokens_validos = 1 - batch['special_tokens_mask'].to(device)
        # Expande a máscara criando uma terceira dimensão (vocab_size) 
        # e colocando do mesmo tamanho que os logits (batch_size, x, vocab_size):
        mask = mask_tokens_validos.unsqueeze(-1).expand(logits.size())

        # Calcula a saída (os pesos wj)
        if param['agg'] == 'sum':
          wj = torch.sum(torch.log(1 + relu(logits*mask)), dim=1)
        else:
          wj, _ = torch.max(torch.log(1 + relu(logits*mask)), dim=1)

        # Antes eu estava retornando wj.to_sparse() pois estava salvando
        # a matriz em disco. Pra salvar num índice invertido não tem mais
        # necessidade disso.

        # Callback
        idx_inicio = i_batch * dataloader.batch_size
        idx_fim = idx_inicio + min(dataloader.batch_size, wj.size()[0])
        indices_tratados = list(range(idx_inicio, idx_fim))
        func_executar_apos_batch(indices_tratados, wj)

Testas os mesmos textos anteriores:

In [93]:
textos = ['where eat pizza', 'what about the weather today']
dataset_textos = Dataset(tokenizer, textos, 256)

Primeiro, com um batch de tamanho 2 (vai inserir padding no primeiro elemento)

In [94]:
# Os input_ids com a máscara:
dataset_textos[0:2]

{'input_ids': [[101, 2073, 4521, 10733, 102, 0, 0], [101, 2054, 2055, 1996, 4633, 2651, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1]], 'special_tokens_mask': [[1, 0, 0, 0, 1, 1, 1], [1, 0, 0, 0, 0, 0, 1]]}

In [95]:
batch_size = 2
dataloader_textos = DataLoader(dataset_textos, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

def callback_imprime(idx_batch, wj):
  nonzero = wj.nonzero()

  for i in range(wj.size()[0]):
    idx_linha_i = nonzero[:, 0] == i
    idx_token_em_wj_i = nonzero[idx_linha_i, 1] # além de ser o índice na matriz wj, é tb o id do token
    val_token_em_wj_i = wj[i, idx_token_em_wj_i]

    tokens = converte_token_ids_para_tokens(tokenizer, idx_token_em_wj_i.tolist())
    print(len(tokens))
    print(list(zip(tokens, val_token_em_wj_i)))

representacao_esparsa_dataloader(model, tokenizer, dataloader_textos, callback_imprime)

  0%|          | 0/1 [00:00<?, ?it/s]

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


52
[('that', tensor(0.0597, device='cuda:0')), ('where', tensor(0.5860, device='cuda:0')), ('city', tensor(0.0860, device='cuda:0')), ('don', tensor(0.2042, device='cuda:0')), ('place', tensor(0.8375, device='cuda:0')), ('best', tensor(0.0039, device='cuda:0')), ('york', tensor(0.2106, device='cuda:0')), ('church', tensor(0.0251, device='cuda:0')), ('country', tensor(0.7220, device='cuda:0')), ('find', tensor(0.0597, device='cuda:0')), ('food', tensor(0.8417, device='cuda:0')), ('places', tensor(0.5735, device='cuda:0')), ('culture', tensor(0.1553, device='cuda:0')), ('location', tensor(1.4597, device='cuda:0')), ('famous', tensor(0.3957, device='cuda:0')), ('italy', tensor(0.6322, device='cuda:0')), ('hotel', tensor(0.4550, device='cuda:0')), ('variety', tensor(0.0327, device='cuda:0')), ('joe', tensor(0.0068, device='cuda:0')), ('store', tensor(0.1994, device='cuda:0')), ('kitchen', tensor(0.5860, device='cuda:0')), ('garden', tensor(0.1800, device='cuda:0')), ('farm', tensor(0.2823,

Agora faz com batch_size = 1, tem que convergir pro caso de chamar cada um separadamente:

In [96]:
# Não tem que ter padding
print(dataset_textos[0])
print(dataset_textos[1])

batch_size = 1
dataloader_textos = DataLoader(dataset_textos, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

representacao_esparsa_dataloader(model, tokenizer, dataloader_textos, callback_imprime)

{'input_ids': [101, 2073, 4521, 10733, 102], 'token_type_ids': [0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1], 'special_tokens_mask': [1, 0, 0, 0, 1]}
{'input_ids': [101, 2054, 2055, 1996, 4633, 2651, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1], 'special_tokens_mask': [1, 0, 0, 0, 0, 0, 1]}


  0%|          | 0/2 [00:00<?, ?it/s]

52
[('that', tensor(0.0597, device='cuda:0')), ('where', tensor(0.5860, device='cuda:0')), ('city', tensor(0.0860, device='cuda:0')), ('don', tensor(0.2042, device='cuda:0')), ('place', tensor(0.8375, device='cuda:0')), ('best', tensor(0.0039, device='cuda:0')), ('york', tensor(0.2106, device='cuda:0')), ('church', tensor(0.0251, device='cuda:0')), ('country', tensor(0.7220, device='cuda:0')), ('find', tensor(0.0597, device='cuda:0')), ('food', tensor(0.8417, device='cuda:0')), ('places', tensor(0.5735, device='cuda:0')), ('culture', tensor(0.1553, device='cuda:0')), ('location', tensor(1.4597, device='cuda:0')), ('famous', tensor(0.3957, device='cuda:0')), ('italy', tensor(0.6322, device='cuda:0')), ('hotel', tensor(0.4550, device='cuda:0')), ('variety', tensor(0.0327, device='cuda:0')), ('joe', tensor(0.0068, device='cuda:0')), ('store', tensor(0.1994, device='cuda:0')), ('kitchen', tensor(0.5860, device='cuda:0')), ('garden', tensor(0.1800, device='cuda:0')), ('farm', tensor(0.2823,

### Dataset e Dataloader TREC-COVID

In [97]:
%%time
ids_trec_covid, textos_trec_covid = zip(*corpus_trec_covid)

dataset_trec_covid = Dataset(tokenizer, textos_trec_covid, 256)
dataloader_trec_covid = DataLoader(dataset_trec_covid, batch_size=batch_size_trec_covid, shuffle=False, collate_fn=collate_fn)

CPU times: user 2.74 s, sys: 47.3 ms, total: 2.78 s
Wall time: 2.74 s


### Classe para índice invertido SPLADE

In [98]:
from collections import Counter
import array
import pickle
import math

# Definição de uma classe para índice invertido
class IndiceInvertidoSplade:

  def __init__(self):
    # Cria um índice invertido vazio
    self.indice = {}

  def adiciona_docs(self, ids_docs, wjs_docs):
    nonzero = wjs_docs.nonzero()

    for i in range(wjs_docs.size()[0]):
      idx_linha_i = nonzero[:, 0] == i # i'ésimo doc do batch
      idx_token_em_wj_i = nonzero[idx_linha_i, 1] # além de ser o índice na matriz wj, é tb o id do token
      val_token_em_wj_i = wjs_docs[i, idx_token_em_wj_i]

      self.adiciona_doc(ids_docs[i], idx_token_em_wj_i, val_token_em_wj_i)

  def adiciona_doc(self, id_doc, idx_tokens, wj_tokens):
    for id, wj in zip(idx_tokens.tolist(), wj_tokens.tolist()):
      self.indice.setdefault(id, {"id_doc": [], "wj": array.array("f", [])})['id_doc'].append(id_doc)
      self.indice.setdefault(id, {"id_doc": [], "wj": array.array("f", [])})['wj'].append(wj)
    
  def pesquisar(self, wjs_query, splade='v1'):
    # Guarda um dicionário onde a chave é o id do documento e o valor é o score desse documento para a query pesquisada
    docs_retornado_com_score = Counter({})

    # Faz a pesquisa de documentos. Para isso iteramos todos os tokens da query
    wjs_query = wjs_query.coalesce()
    for id_token_query, wj_do_token_na_query in zip(wjs_query.indices()[0].tolist(), wjs_query.values().tolist()):
      # É possível que a query contenha algum termo que não foi indexado. Se isso ocorrer, apenas pula o termo
      if id_token_query not in self.indice:
        continue

      # Pega a lista de documentos que será analisado
      docs_que_tem_token = self.indice[id_token_query]['id_doc']
      wj_do_token_nos_docs = self.indice[id_token_query]['wj']

      # Agora já temos calculado o score de todos os documentos desse token. Só adiciona ao acumulador de score atual
      # docs_retornado_com_score += score_dos_docs_deste_token -> Se fosse usar dict direto no índice seria assim, mas a memória não está aguentando guardar os scores de ambos
      multiplicador_token_query = wj_do_token_na_query if splade == 'v1' else 1

      for id_doc, wj_do_token_no_doc in zip(docs_que_tem_token, wj_do_token_nos_docs):
        docs_retornado_com_score[id_doc] += wj_do_token_no_doc * multiplicador_token_query
      
    # Agora converte esse dict em uma lista de tuplas com a chave (id_doc) e valor (score_do_doc)
    docs_com_score = list(docs_retornado_com_score.items())

    # E ordena do mais relevante para o menos relevante
    return sorted(docs_com_score, key=lambda x: x[1], reverse=True)

In [99]:
%%time
idx_splade = IndiceInvertidoSplade()

def popular_indice_invertido(idx_batch, wj_batch):
  ids_doc_batch = [ids_trec_covid[i] for i in idx_batch]
  idx_splade.adiciona_docs(ids_doc_batch, wj_batch)

def salvar_indice(idx_splade):
  nome_arquivo_pickle = param["nome_arquivo_indice_invertido_docs"]
  diretorio_destino_cp = f"'{dir_aula_7}'"
  with open(nome_arquivo_pickle, 'wb') as f:
    pickle.dump(idx_splade.indice, f)
  !cp {nome_arquivo_pickle} {diretorio_destino_cp}

def recuperar_indice(idx_splade):
  with open(arq_indice_pickle, 'rb') as f:
    idx_splade.indice = pickle.load(f)

arq_indice_pickle = f'{dir_aula_7}{param["nome_arquivo_indice_invertido_docs"]}'

if gerar_indice_invertido_docs:
  representacao_esparsa_dataloader(model, tokenizer, dataloader_trec_covid, popular_indice_invertido)    
  salvar_indice(idx_splade)
else:
  recuperar_indice(idx_splade)

  0%|          | 0/5355 [00:00<?, ?it/s]

CPU times: user 19min 35s, sys: 5.19 s, total: 19min 40s
Wall time: 19min 33s


### Testes no TREC-COVID

In [100]:
def run_all_queries_indice_invertido_splade(file, model, tokenizer, idx_splade, splade='v1'):
  print('Carregando as queries do arquivo queries.jsonl...\n')
  queries_trec_covid = carrega_queries_trec_covid()

  print(f'Total de queries que serão avaliadas: {len(queries_trec_covid)}')
  cnt = 0
  with open(file, 'w') as runfile:
    for query in queries_trec_covid:
      id = query['id']
      texto = query['texto']

      wj_query = representacao_esparsa_do_texto(model, tokenizer, texto, True, param['manter_contribuicao_CLS_SEP_da_matriz_doc'])
      if cnt % 10 == 0:
        print(f'{cnt} queries completadas')

      # Usa o índice invertido pra pesquisar
      docs_score = idx_splade.pesquisar(wj_query, splade)

      for i in range(0, min(1000, len(docs_score))): # Pega os primeiros 1000 resultados
        _ = runfile.write('{} Q0 {} {} {:.6f} BM_25\n'.format(id, docs_score[i][0], i+1, docs_score[i][1]))

      cnt += 1
    print(f'{cnt} queries completadas')

In [101]:
%%time
run_all_queries_indice_invertido_splade('run-splade-iidx-v1.txt', model, tokenizer, idx_splade, 'v1')
!python -m pyserini.eval.trec_eval -c -m ndcg_cut.10 collections/trec-covid/qrels/test_corrigido.tsv run-splade-iidx-v1.txt #type: ign

Carregando as queries do arquivo queries.jsonl...

Total de queries que serão avaliadas: 50
0 queries completadas
10 queries completadas
20 queries completadas
30 queries completadas
40 queries completadas
50 queries completadas
/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar already exists!
Skipping download.
Running command: ['java', '-jar', '/root/.cache/pyserini/eval/jtreceval-0.0.5-jar-with-dependencies.jar', '-c', '-m', 'ndcg_cut.10', 'collections/trec-covid/qrels/test_corrigido.tsv', 'run-splade-iidx-v1.txt']
Results:
ndcg_cut_10           	all	0.7269
CPU times: user 55.9 s, sys: 458 ms, total: 56.3 s
Wall time: 1min


In [102]:
%%time
## Roda as 50 queries sem fazer mais nada, só pra contar o tempo de execução pra rodar as queries
for query in queries_trec_covid:
  wj_query = representacao_esparsa_do_texto(model, tokenizer, query['texto'], True, param['manter_contribuicao_CLS_SEP_da_matriz_doc'])
  docs_score = idx_splade.pesquisar(wj_query, 'v1')

CPU times: user 55.6 s, sys: 227 ms, total: 55.9 s
Wall time: 54.9 s
